In [19]:
import numpy as np
import pandas as pd
from ase import Atoms
from ase import neighborlist
from ase import units
from ase.io import read, write
from ase.io.trajectory import Trajectory
from ase.md.verlet import VelocityVerlet
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.io.extxyz import read_extxyz, write_extxyz
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [20]:
pred_step = 4
input = 6
file = r"Data\EtOH_moldyn_single.xyz"

#The function takes arrays of forces and returns a VAR OLS model
#_____Inputs______
# main - 1D np array with the time series which the model fits to.
# secondary - 2D np array with the secondary time series used to predict the main time series
# order - Positive integer which decides the numper of points in the past is used to predict the next point
#
#_____Output______
# Returns a OLS model that uses the main and secondary time series to predict the main time series. In order to predict t+2 of the main timeseries,
# t+1 of both main and secondary time series must be known. If the values of the secondary time series is not known this can be done by also creating models
# for the secondary time series and use those to predict the value at t+1
def create_VAR_OLS_model(main, secondary, order=2):
    input = len(main)-order
    num_secondaries = secondary.shape[0]
    X = np.zeros([input,order*(num_secondaries+1)+1])
    for i in range(order):
        X[:,i*(num_secondaries+1)] = main[i:input+i]
        for j in range(num_secondaries):
            X[:,i*(num_secondaries+1)+j+1] = secondary[j,i:input+i]
    X[:,-1] = np.ones(len(main)-order)
    return sm.OLS(main[order:], X)

# Returns 3 fitted models from create_VAR_OLS_model to create 3 models to respectively predict the forces in x, y, and z direction for an atom in a molecule.
# Uses L2 regularized fitting.
#_____Inputs______
# forces - 2D np array with the three force time series for the atom chosen. Has the shape [N, 3] where N is the number of input points in the model.
# order - Positive integer which decides the numper of points in the past is used to predict the next point
#
#_____Output______
# Returns 3 fitted VAR OLS which can be used to predict future points in the time series. t+1 must be predicted for all three time series before t+2 can be predicted.
def create_atom_OLS_model(forces, order=2):
    model_x = create_VAR_OLS_model(forces[:,0], forces[:,1:], order=order).fit_regularized(l1_wt=0)
    model_y = create_VAR_OLS_model(forces[:,1], forces[:,[0,2]], order=order).fit_regularized(l1_wt=0)
    model_z = create_VAR_OLS_model(forces[:,2], forces[:,:2], order=order).fit_regularized(l1_wt=0)
    return model_x, model_y, model_z

# Function which predicts the forces on the atoms in a molecule for a certain number of time steps
#_____Inputs______
# file - String with the name of the xyz file containing information on the molecule
# input - Positive integer with the number of data points used as input in training the OLS VAR models
# pred_step - Positive integer which decides the number of time steps predicted by the OLS VAR models
# order - Positive integer which decides the numper of points in the past is used to predict the next point
#
#_____Output______
# Returns a 3D np array with the predicted forces. The array as the following shape [N, 3, p], with N being the number of atoms in the molecule
# and p being the number of time steps predicted
def predict_forces(file=file, input=input, pred_step=pred_step, order=2):
    mol = read(file, index=slice(-input, None))
    n_atoms = mol[0].get_global_number_of_atoms()
    predicted_forces = np.zeros((n_atoms, 3, pred_step))
    forces = np.zeros((n_atoms, 3, input))

    for i in range(len(mol)):
        forces[:,:,i] = mol[i].get_forces()
    for i in range(n_atoms):
        model_x, model_y, model_z = create_atom_OLS_model(forces[:,:,:input], order=order)
        forecast_forces = np.zeros((3, pred_step+order))
        forecast_forces[:,:order] = forces[i,:,-order:]
        for j in range(pred_step):
            input_x = np.array([forecast_forces[0,j], forecast_forces[1,j], forecast_forces[2,j],forecast_forces[0,j+1], forecast_forces[1,j+1], forecast_forces[2,j+1], 1])
            input_y = np.array([forecast_forces[1,j], forecast_forces[0,j], forecast_forces[2,j],forecast_forces[1,j+1], forecast_forces[0,j+1], forecast_forces[2,j+1], 1])
            input_z = np.array([forecast_forces[2,j], forecast_forces[0,j], forecast_forces[1,j],forecast_forces[2,j+1], forecast_forces[0,j+1], forecast_forces[1,j+1], 1])
            forecast_forces[0,order+j] = model_x.predict(input_x)
            forecast_forces[1,order+j] = model_y.predict(input_y)
            forecast_forces[2,order+j] = model_z.predict(input_z)
        predicted_forces[i,:,:] = forecast_forces[:,2:]
        
    return predicted_forces

